In [27]:
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import timedelta

# from cartoframes import read_carto, to_carto, has_table, list_tables
# from cartoframes.auth import set_default_credentials, Credentials
# #from cartoframes.viz import Map, Layer, basemaps
# from cartoframes.data.clients import SQLClient
from shapely.geometry import Point, LineString
from zipfile import ZipFile

In [28]:
header = list(pd.read_csv('TripBulkReportTripsHeaders.csv',keep_default_na=True))
header.append('Unnamed 31')
header.append('Unnamed 32')
trip_df = pd.read_csv('new_trips.csv', keep_default_na=True, names=header)

In [18]:
trip_df.head()

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MovementType,OriginCbg,DestCbg,StartTimezone,EndTimezone,WaypointFreqSec,StartQk,EndQk,Unnamed 31,Unnamed 32
0,4e6d3859173a783b35f234126e7d84ab,0da5283cac35fa6de9a83477d1477377,d07e70efcfab08731a97e7b91be644de,1,2020-12-28T14:49:16.000Z,1,2020-12-28T15:53:22.000Z,1,38.71498,-104.70176,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002301310,23101210113102230,NaN,NaN
1,a9f9b922d560178ab3bd1b773fef437d,0ec14c96079ca0e5b6afd3d0a429cf49,42998cf32d552343bc8e460416382dca,1,2020-12-22T20:15:06.000Z,2,2020-12-22T20:26:22.000Z,2,38.74497,-104.79377,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210113103011,23101211000023121,NaN,NaN
2,f6aec89e2dd3cd8fe5d09e45fc083de9,13d0412d930e3669ac09040c767f1097,42998cf32d552343bc8e460416382dca,1,2020-12-22T01:32:52.000Z,1,2020-12-22T01:59:33.000Z,1,38.76318,-104.80476,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111320233,23101210111320232,NaN,NaN
3,b87c20dbcc739db4ff64c100398437b7,1520526e3b9c71175574099508704611,d07e70efcfab08731a97e7b91be644de,1,2020-12-22T14:13:19.000Z,2,2020-12-22T14:30:36.000Z,2,38.69890,-104.68940,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002330133,23101210113101210,NaN,NaN
4,ea65f3a1f58f2431dcfe317be0bdd002,15c0082ebc282eaf14a6c80c67017142,66f041e16a60928b05a7e228a89c3799,1,2020-12-22T11:46:26.000Z,2,2020-12-22T12:03:42.000Z,2,38.75568,-104.80339,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111322320,23101210113121113,NaN,NaN


In [19]:
number_of_dup = len(trip_df[trip_df.duplicated(subset=list(trip_df.columns).remove('TripId')) == True])
number_of_fort = len(trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')])

print('Number of rows in original data:', len(trip_df))
print('Number of data in Fort Carson : ',number_of_fort)
print('Number of duplicated rows : ',number_of_dup)

Number of rows in original data: 2316130
Number of data in Fort Carson :  2316130
Number of duplicated rows :  0


In [29]:
trip_df = trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

In [30]:
gdf = gpd.GeoDataFrame(trip_df)

In [31]:
gdf['MST_start_datetime'] = pd.to_datetime(gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
gdf['MST_end_datetime'] = pd.to_datetime(gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

gdf['MST_start_date'] = gdf['MST_start_datetime'].apply(lambda x:x.date())
gdf['MST_start_time'] = gdf['MST_start_datetime'].apply(lambda x:x.time())

gdf['MST_end_date'] = gdf['MST_end_datetime'].apply(lambda x:x.date())
gdf['MST_end_time'] = gdf['MST_end_datetime'].apply(lambda x:x.time())

In [32]:
# create df of start day, month, year 
startYear = list(pd.DatetimeIndex(gdf['MST_start_date']).year)
startMonth = list(pd.DatetimeIndex(gdf['MST_start_date']).month)
startDay = list(pd.DatetimeIndex(gdf['MST_start_date']).day)

# insert the start month, day, year column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+1), 'MST_start_month',startMonth)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+2), 'MST_start_day',startDay)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+3), 'MST_start_year',startYear)

# create df of start hour, minutes, and seconds
startHour = list(pd.DatetimeIndex(gdf['MST_start_datetime']).hour)
startMin = list(pd.DatetimeIndex(gdf['MST_start_datetime']).minute)
startSec = list(pd.DatetimeIndex(gdf['MST_start_datetime']).second)

# insert the start hour, minutes, seconds column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+1), 'MST_start_hour',startHour)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+2), 'MST_start_minute',startMin)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+3), 'MST_start_second',startSec)

# create df of end day, month, year 
endYear = list(pd.DatetimeIndex(gdf['MST_end_date']).year)
endMonth = list(pd.DatetimeIndex(gdf['MST_end_date']).month)
endDay = list(pd.DatetimeIndex(gdf['MST_end_date']).day)

# insert the start month, day, year column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+1), 'MST_end_month',endMonth)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+2), 'MST_end_day',endDay)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+3), 'MST_end_year',endYear)

# create df of end hour, minutes, and seconds
endHour = list(pd.DatetimeIndex(gdf['MST_end_datetime']).hour)
endMin = list(pd.DatetimeIndex(gdf['MST_end_datetime']).minute)
endSec = list(pd.DatetimeIndex(gdf['MST_end_datetime']).second)


# insert the start hour, minutes, seconds column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+1), 'MST_end_hour',endHour)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+2), 'MST_end_minute',endMin)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+3), 'MST_end_second',endSec)


In [41]:
gdf.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Startlocation', 'Endlocation', 'MST_start_datetime',
       'MST_end_datetime', 'MST_start_date', 'MST_start_month',
       'MST_start_day', 'MST_start_year', 'MST_start_time', 'MST_start_hour',
       'MST_start_minute', 'MST_start_second', 'MST_end_date', 'MST_end_month',
       'MST_end_day', 'MST_end_year', 'MST_end_time', 'MST_end_hour',
       'MST_end_minute', 'MST_end_second', 'day_of_week', 'start_year_month'],
      dtype='object')

In [35]:
gdf['MST_start_datetime'].unique()

<DatetimeArray>
[       '2020-12-28 07:49:16-07:00',        '2020-12-22 13:15:06-07:00',
        '2020-12-21 18:32:52-07:00',        '2020-12-22 07:13:19-07:00',
        '2020-12-22 04:46:26-07:00',        '2020-12-28 15:44:28-07:00',
        '2020-12-22 08:37:10-07:00',        '2020-12-22 16:03:37-07:00',
        '2020-12-27 13:09:25-07:00',        '2020-12-23 18:09:33-07:00',
 ...
        '2021-01-20 12:11:56-07:00',        '2021-01-20 09:38:57-07:00',
        '2021-01-22 07:35:37-07:00',        '2021-01-21 15:27:38-07:00',
        '2021-01-21 04:50:34-07:00',        '2021-01-19 06:02:49-07:00',
        '2021-01-19 07:26:07-07:00',        '2021-01-22 11:18:13-07:00',
        '2021-01-22 16:05:57-07:00', '2021-01-21 07:08:04.511000-07:00']
Length: 2173523, dtype: datetime64[ns, MST]

In [12]:
len(gdf['TripId'].unique())

2316130

In [42]:
gdf_202010_to_202009_new = gdf.loc[(gdf['start_year_month']!='2020-09') & (gdf['start_year_month']!='2020-08')]

In [43]:
gdf['day_of_week'] = gdf['MST_start_datetime'].dt.day_name()

gdf['start_year_month'] = pd.to_datetime(gdf['MST_start_datetime']).dt.to_period('M')

/Users/cindyhuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [53]:
np.sort(workday[pd.to_datetime(workday['MST_start_datetime']).dt.day_name()=='Monday']['MST_start_date'].unique())

array([datetime.date(2020, 10, 5), datetime.date(2020, 10, 12),
       datetime.date(2020, 10, 19), datetime.date(2020, 10, 26),
       datetime.date(2020, 11, 2), datetime.date(2020, 11, 9),
       datetime.date(2020, 11, 16), datetime.date(2020, 11, 23),
       datetime.date(2020, 11, 30), datetime.date(2020, 12, 7),
       datetime.date(2020, 12, 14), datetime.date(2020, 12, 21),
       datetime.date(2020, 12, 28), datetime.date(2021, 1, 4),
       datetime.date(2021, 1, 11), datetime.date(2021, 1, 18),
       datetime.date(2021, 1, 25), datetime.date(2021, 2, 1),
       datetime.date(2021, 2, 8), datetime.date(2021, 2, 15),
       datetime.date(2021, 2, 22), datetime.date(2021, 3, 1),
       datetime.date(2021, 3, 8), datetime.date(2021, 3, 15),
       datetime.date(2021, 3, 22), datetime.date(2021, 3, 29),
       datetime.date(2021, 4, 5), datetime.date(2021, 4, 12),
       datetime.date(2021, 4, 19), datetime.date(2021, 4, 26),
       datetime.date(2021, 5, 3), datetime.date(2021

In [44]:
print('Total number of trips between 2020/10 and 2021/09:',len(gdf_202010_to_202009_new))

Total number of trips between 2020/10 and 2021/09: 2155164


In [54]:
np.sort(workday.groupby(by='StartWDay')['MST_start_date'].unique().iloc[0])

array([datetime.date(2020, 10, 5), datetime.date(2020, 10, 12),
       datetime.date(2020, 10, 19), datetime.date(2020, 10, 26),
       datetime.date(2020, 11, 2), datetime.date(2020, 11, 9),
       datetime.date(2020, 11, 16), datetime.date(2020, 11, 23),
       datetime.date(2020, 11, 30), datetime.date(2020, 12, 7),
       datetime.date(2020, 12, 14), datetime.date(2020, 12, 21),
       datetime.date(2020, 12, 28), datetime.date(2021, 1, 4),
       datetime.date(2021, 1, 11), datetime.date(2021, 1, 18),
       datetime.date(2021, 1, 25), datetime.date(2021, 2, 1),
       datetime.date(2021, 2, 8), datetime.date(2021, 2, 15),
       datetime.date(2021, 2, 22), datetime.date(2021, 3, 1),
       datetime.date(2021, 3, 8), datetime.date(2021, 3, 15),
       datetime.date(2021, 3, 22), datetime.date(2021, 3, 29),
       datetime.date(2021, 4, 5), datetime.date(2021, 4, 12),
       datetime.date(2021, 4, 19), datetime.date(2021, 4, 26),
       datetime.date(2021, 5, 3), datetime.date(2021

In [45]:
gdf_202010_to_202009_new['day_of_week'].unique()

array(['Monday', 'Tuesday', 'Sunday', 'Wednesday', 'Saturday', 'Thursday',
       'Friday'], dtype=object)

In [46]:
workday = gdf_202010_to_202009_new[(gdf_202010_to_202009_new['day_of_week']!='Saturday') &(gdf_202010_to_202009_new['day_of_week']!='Sunday') ]

old_data_workday = old_large_202010_to_202009[(old_large_202010_to_202009['day_of_week']!='Saturday') &(old_large_202010_to_202009['day_of_week']!='Sunday') ]

In [47]:
print('Total number of Trips on weekday by each Geospatial Type :')
workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Trips on weekday by each Geospatial Type :


GeospatialType
EI    491086
IE    490023
II    912971
Name: TripId, dtype: int64

In [48]:
number_of_EI_weekday = workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(workday['MST_start_date'].unique())
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('On average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

Total number of EI trips in weekdays: 491086
Total number of weekdays: 261
On average, the number of EI trips per day on weekdays is: 1881.6


In [49]:
print('Number of days for each workday:')
workday.groupby(by='day_of_week')['MST_start_date'].nunique()

Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    52
Name: MST_start_date, dtype: int64

In [51]:
number_of_EI_each_weekday = workday.loc[workday['GeospatialType']=='EI'].groupby(['day_of_week']).count()['TripId']


number_of_days = workday.groupby(by='day_of_week')['MST_start_date'].nunique()

daily_weekday_average = pd.concat([number_of_EI_each_weekday,number_of_days],axis=1).set_index(np.array(['Mon','Tue','Wed','Thurs','Fri'])).\
rename(columns={'TripId':'number of trips','MST_start_date':'number of days'})
daily_weekday_average['average number of trips'] = round(daily_weekday_average['number of trips']/daily_weekday_average['number of days'],1)

In [174]:
number_of_EI_each_weekday = old_data_workday.loc[old_data_workday['GeospatialType']=='EI'].groupby(['day_of_week']).count()['TripId']


number_of_days = old_data_workday.groupby(by='day_of_week')['MST_start_date'].nunique()

old_daily_weekday_average = pd.concat([number_of_EI_each_weekday,number_of_days],axis=1).set_index(np.array(['Mon','Tue','Wed','Thurs','Fri'])).\
rename(columns={'TripId':'number of trips','MST_start_date':'number of days'})
old_daily_weekday_average['average number of trips'] = round(old_daily_weekday_average['number of trips']/old_daily_weekday_average['number of days'],1)

In [175]:
print('Old data daily weekly average:\n')
daily_weekday_average

Old data daily weekly average:



,number of trips,number of days,average number of trips
Mon,95015,52,1827.2
Tue,90744,52,1745.1
Wed,102067,53,1925.8
Thurs,100118,52,1925.3
Fri,103142,52,1983.5


In [176]:
print('New data daily weekly average:\n')
daily_weekday_average

New data daily weekly average:



,number of trips,number of days,average number of trips
Mon,95015,52,1827.2
Tue,90744,52,1745.1
Wed,102067,53,1925.8
Thurs,100118,52,1925.3
Fri,103142,52,1983.5


In [177]:
# gdf_202010_to_202009_new.head()

## Old data investigation

In [121]:
header = list(pd.read_csv('TripBulkReportTripsHeaders.csv',keep_default_na=True))
header.append('Unnamed 31')
header.append('Unnamed 32')
trip_df = pd.read_csv('trips (4).csv', keep_default_na=True, names=header)
small_trips_df = pd.read_csv('trips.csv1', keep_default_na=True, names=header)

frames = [trip_df,small_trips_df]
old_data_gdf = pd.concat(frames)

In [122]:
old_data_gdf.head()

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MovementType,OriginCbg,DestCbg,StartTimezone,EndTimezone,WaypointFreqSec,StartQk,EndQk,Unnamed 31,Unnamed 32
0,4e6d3859173a783b35f234126e7d84ab,0da5283cac35fa6de9a83477d1477377,d07e70efcfab08731a97e7b91be644de,1,2020-12-28T14:49:16.000Z,1,2020-12-28T15:53:22.000Z,1,38.71498,-104.70176,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002301310,23101210113102230,NaN,NaN
1,a9f9b922d560178ab3bd1b773fef437d,0ec14c96079ca0e5b6afd3d0a429cf49,42998cf32d552343bc8e460416382dca,1,2020-12-22T20:15:06.000Z,2,2020-12-22T20:26:22.000Z,2,38.74497,-104.79377,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210113103011,23101211000023121,NaN,NaN
2,f6aec89e2dd3cd8fe5d09e45fc083de9,13d0412d930e3669ac09040c767f1097,42998cf32d552343bc8e460416382dca,1,2020-12-22T01:32:52.000Z,1,2020-12-22T01:59:33.000Z,1,38.76318,-104.80476,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111320233,23101210111320232,NaN,NaN
3,b87c20dbcc739db4ff64c100398437b7,1520526e3b9c71175574099508704611,d07e70efcfab08731a97e7b91be644de,1,2020-12-22T14:13:19.000Z,2,2020-12-22T14:30:36.000Z,2,38.69890,-104.68940,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002330133,23101210113101210,NaN,NaN
4,ea65f3a1f58f2431dcfe317be0bdd002,15c0082ebc282eaf14a6c80c67017142,66f041e16a60928b05a7e228a89c3799,1,2020-12-22T11:46:26.000Z,2,2020-12-22T12:03:42.000Z,2,38.75568,-104.80339,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111322320,23101210113121113,NaN,NaN


In [123]:
old_data_gdf = old_data_gdf[(old_data_gdf['OriginZoneName']!='Buc-ee\'s Giddings TX')& (old_data_gdf['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
old_data_gdf['Startlocation'] = list(zip(old_data_gdf.StartLocLat, old_data_gdf.StartLocLon))
old_data_gdf['Startlocation'] = old_data_gdf['Startlocation'].apply(Point)

old_data_gdf['Endlocation'] = list(zip(old_data_gdf.EndLocLat, old_data_gdf.EndLocLon))
old_data_gdf['Endlocation'] = old_data_gdf['Endlocation'].apply(Point)

old_data_gdf= old_data_gdf.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

In [124]:
old_data_gdf = gpd.GeoDataFrame(old_data_gdf)

In [126]:
old_data_gdf['MST_start_datetime'] = pd.to_datetime(old_data_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
old_data_gdf['MST_end_datetime'] = pd.to_datetime(old_data_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

old_data_gdf['MST_start_date'] = old_data_gdf['MST_start_datetime'].apply(lambda x:x.date())
old_data_gdf['MST_start_time'] = old_data_gdf['MST_start_datetime'].apply(lambda x:x.time())

old_data_gdf['MST_end_date'] = old_data_gdf['MST_end_datetime'].apply(lambda x:x.date())
old_data_gdf['MST_end_time'] = old_data_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [129]:
old_data_gdf['day_of_week'] = old_data_gdf['MST_start_datetime'].dt.day_name()

In [128]:
old_data_gdf.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Startlocation', 'Endlocation', 'MST_start_datetime',
       'MST_end_datetime', 'MST_start_date', 'MST_start_time', 'MST_end_date',
       'MST_end_time'],
      dtype='object')

In [131]:
old_data_gdf['day_of_week']

0           Monday
1          Tuesday
2           Monday
3          Tuesday
4          Tuesday
            ...   
1160367     Friday
1160368     Friday
1160369     Monday
1160370    Tuesday
1160371    Tuesday
Name: day_of_week, Length: 3476502, dtype: object

In [138]:
old_data_gdf['start_year_month'] = pd.to_datetime(old_data_gdf['MST_start_datetime']).dt.to_period('M')


/Users/cindyhuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [142]:
trip_df = trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

trip_gdf = gpd.GeoDataFrame(trip_df)

large_trip_gdf = trip_gdf

large_trip_gdf['MST_start_datetime'] = pd.to_datetime(large_trip_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
large_trip_gdf['MST_end_datetime'] = pd.to_datetime(large_trip_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

large_trip_gdf['MST_start_date'] = large_trip_gdf['MST_start_datetime'].apply(lambda x:x.date())
large_trip_gdf['MST_start_time'] = large_trip_gdf['MST_start_datetime'].apply(lambda x:x.time())

large_trip_gdf['MST_end_date'] = large_trip_gdf['MST_end_datetime'].apply(lambda x:x.date())
large_trip_gdf['MST_end_time'] = large_trip_gdf['MST_end_datetime'].apply(lambda x:x.time())

large_trip_gdf['day_of_week'] = large_trip_gdf['MST_start_datetime'].dt.day_name()



In [146]:
large_trip_gdf['start_year_month'] = pd.to_datetime(large_trip_gdf['MST_start_datetime']).dt.to_period('M')


/Users/cindyhuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [143]:
small_trips_df = small_trips_df[(small_trips_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (small_trips_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
small_trips_df['Startlocation'] = list(zip(small_trips_df.StartLocLat, small_trips_df.StartLocLon))
small_trips_df['Startlocation'] = small_trips_df['Startlocation'].apply(Point)

small_trips_df['Endlocation'] = list(zip(small_trips_df.EndLocLat, small_trips_df.EndLocLon))
small_trips_df['Endlocation'] = small_trips_df['Endlocation'].apply(Point)

small_trips_df= small_trips_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

small_trips_df = gpd.GeoDataFrame(small_trips_df)

small_trip_gdf = small_trips_df

small_trip_gdf['MST_start_datetime'] = pd.to_datetime(small_trip_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
small_trip_gdf['MST_end_datetime'] = pd.to_datetime(small_trip_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

small_trip_gdf['MST_start_date'] = small_trip_gdf['MST_start_datetime'].apply(lambda x:x.date())
small_trip_gdf['MST_start_time'] = small_trip_gdf['MST_start_datetime'].apply(lambda x:x.time())

small_trip_gdf['MST_end_date'] = small_trip_gdf['MST_end_datetime'].apply(lambda x:x.date())
small_trip_gdf['MST_end_time'] = small_trip_gdf['MST_end_datetime'].apply(lambda x:x.time())

small_trip_gdf['day_of_week'] = small_trip_gdf['MST_start_datetime'].dt.day_name()

In [147]:
small_trip_gdf['start_year_month'] = pd.to_datetime(small_trip_gdf['MST_start_datetime']).dt.to_period('M')


In [182]:
old_gdf_202010_to_202009 = old_data_gdf.loc[(old_data_gdf['start_year_month']!='2020-09') & (old_data_gdf['start_year_month']!='2020-08')]
old_large_202010_to_202009 = large_trip_gdf.loc[(large_trip_gdf['start_year_month']!='2020-09') & (large_trip_gdf['start_year_month']!='2020-08')]
old_small_202010_to_202009 = small_trip_gdf.loc[(small_trip_gdf['start_year_month']!='2020-09') & (small_trip_gdf['start_year_month']!='2020-08')]
small_start = small_trip_gdf['MST_start_date'].min()
small_end = small_trip_gdf['MST_start_date'].max()
large_start = large_trip_gdf['MST_start_date'].min()
large_end = large_trip_gdf['MST_start_date'].max()

start = old_gdf_202010_to_202009['MST_start_date'].min()
end = old_gdf_202010_to_202009['MST_start_date'].max()


old_large_start = old_large_202010_to_202009['MST_start_date'].min()
old_large_end= old_large_202010_to_202009['MST_start_date'].max()

old_small_start = old_small_202010_to_202009['MST_start_date'].min()
old_small_end= old_small_202010_to_202009['MST_start_date'].max()

start_new = gdf_202010_to_202009_new['MST_start_date'].min()
end_new = gdf_202010_to_202009_new['MST_start_date'].max()
print('Old trips aggregated dataset time range is between '+str(old_data_gdf['MST_start_date'].min())+ ' and '+str(old_data_gdf['MST_start_date'].max()))
# print('Old trips large dataset time range is between '+str(large_start)+ ' and '+str(large_end))
# print('Old trips subset data time range is between '+str(small_start)+ ' and '+str(small_end))


print('\nNew trips dataset time range is between '+str(gdf['MST_start_date'].min())+ ' and '+str(gdf['MST_start_date'].max()))
      

print('\nSetting time range between 2020-10-01 and 2021-09-30, the total number of trips breakdown is as follows: ')

# print('Old trips large dataset time range is between '+str(old_large_start)+ ' and '+str(old_large_end))
# print('Old trips subset data time range is between '+str(old_small_start)+ ' and '+str(old_small_end))



print('Total numbers of trips in old aggredated data : '+str(len(old_gdf_202010_to_202009['TripId'].unique())))
print('Total numbers of trips in new data : '+str(len(gdf_202010_to_202009_new['TripId'].unique())))
print('Total numbers of trips in old large data : '+str(len(old_large_202010_to_202009['TripId'].unique())))
print('Total numbers of trips in old sub-data : '+str(len(old_small_202010_to_202009['TripId'].unique())))


Old trips aggregated dataset time range is between 2020-08-31 and 2021-09-30

New trips dataset time range is between 2020-08-31 and 2021-09-30

Setting time range between 2020-10-01 and 2021-09-30, the total number of trips breakdown is as follows: 
Total numbers of trips in old aggredated data : 3234574
Total numbers of trips in new data : 2155164
Total numbers of trips in old large data : 2155164
Total numbers of trips in old sub-data : 1079410


In [157]:
old_large_202010_to_202009['weekday_or_weekend'] = old_large_202010_to_202009['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = old_large_202010_to_202009.loc[old_large_202010_to_202009['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(old_large_202010_to_202009['MST_start_date'].unique())

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trips breakdown on old trip data with range between '+str(old_large_start)+ ' and '+str(old_large_end))

weekend_weekday_avg

weekday_avg = weekend_weekday_avg.loc['Weekday']['avg number of trips']
weekend_avg = weekend_weekday_avg.loc['Weekend']['avg number of trips']
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day'.format(total_number_of_EI_trips/number_of_days))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekdays'.format(weekday_avg))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekends'.format(weekend_avg))



<ipython-input-157-bed8d4897212>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_large_202010_to_202009['weekday_or_weekend'] = old_large_202010_to_202009['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')


EI trips breakdown on old trip data with range between 2020-10-01 and 2021-09-30
On average, there are 1,588 trips onto and off of Fort Carson per day
On average, there are 1,882 trips onto and off of Fort Carson per day for weekdays
On average, there are 850 trips onto and off of Fort Carson per day for weekends


In [158]:
gdf_202010_to_202009_new['weekday_or_weekend'] = gdf_202010_to_202009_new['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = gdf_202010_to_202009_new.loc[gdf_202010_to_202009_new['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(gdf_202010_to_202009_new['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trips breakdown on new trip data with range between '+str(start_new)+ ' and '+str(end_new))
weekend_weekday_avg

weekday_avg = weekend_weekday_avg.loc['Weekday']['avg number of trips']
weekend_avg = weekend_weekday_avg.loc['Weekend']['avg number of trips']
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day'.format(total_number_of_EI_trips/number_of_days))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekdays'.format(weekday_avg))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekends'.format(weekend_avg))

<ipython-input-158-d904278541bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_202010_to_202009_new['weekday_or_weekend'] = gdf_202010_to_202009_new['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')


EI trips breakdown on new trip data with range between 2020-10-01 and 2021-09-30
On average, there are 1,588 trips onto and off of Fort Carson per day
On average, there are 1,882 trips onto and off of Fort Carson per day for weekdays
On average, there are 850 trips onto and off of Fort Carson per day for weekends


In [160]:
workday = gdf_202010_to_202009_new[(gdf_202010_to_202009_new['day_of_week']!='Saturday') &(gdf_202010_to_202009_new['day_of_week']!='Sunday') ]



In [173]:
print('Total number of Each Type of Trips in weekday on new data:')
workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Each Type of Trips in weekday on new data:


GeospatialType
EI    491086
IE    490023
II    912971
Name: TripId, dtype: int64

In [172]:
print('Total number of Each Type of Trips in weekday on old data:')
old_data_workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Each Type of Trips in weekday on old data:


GeospatialType
EI    491086
IE    490023
II    912971
Name: TripId, dtype: int64

In [162]:
number_of_EI_weekday = old_data_workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(old_data_workday['MST_start_date'].unique())
print('EI trip break down of old data between 2020-10-01 and 2021-09-30:\n ')
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('\nOn average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

print('Number of days for each workday:')
old_data_workday.groupby(by='day_of_week')['MST_start_date'].nunique()

EI trip break down of old data between 2020-10-01 and 2021-09-30:
 
Total number of EI trips in weekdays: 491086
Total number of weekdays: 261

On average, the number of EI trips per day on weekdays is: 1881.6
Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    52
Name: MST_start_date, dtype: int64

In [163]:
number_of_EI_weekday = workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(workday['MST_start_date'].unique())
print('EI trip break down of new data between 2020-10-01 and 2021-09-30\n ')
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('\nOn average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

print('Number of days for each workday:')
workday.groupby(by='day_of_week')['MST_start_date'].nunique()

EI trip break down of new data between 2020-10-01 and 2021-09-30
 
Total number of EI trips in weekdays: 491086
Total number of weekdays: 261

On average, the number of EI trips per day on weekdays is: 1881.6
Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    52
Name: MST_start_date, dtype: int64

In [168]:
large_trip_gdf['weekday_or_weekend'] = large_trip_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = large_trip_gdf.loc[large_trip_gdf['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(large_trip_gdf['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trip breakdown by weekday/weekend for old data:\n ')

weekend_weekday_avg


EI trip breakdown by weekday/weekend for old data:
 


,number of EI trips,number of days,avg number of trips
weekday_or_weekend,,,
Weekday,528658,284,1861.5
Weekend,94681,112,845.4


In [167]:
gdf['weekday_or_weekend'] = gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = gdf.loc[gdf['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(gdf['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trip breakdown by weekday/weekend for new data:\n ')

weekend_weekday_avg

EI trip breakdown by weekday/weekend for new data:
 


,number of EI trips,number of days,avg number of trips
weekday_or_weekend,,,
Weekday,528658,284,1861.5
Weekend,94681,112,845.4


In [181]:
# start = old_data_gdf['MST_start_date'].min()
# end = old_data_gdf['MST_start_date'].max()
# start_new = gdf_202010_to_202009_new['MST_start_date'].min()
# end_new = gdf_202010_to_202009_new['MST_start_date'].max()
# print('Old trips dataset time range is between '+str(start)+ ' and '+str(end))
# print('New trips dataset time range is between '+str(start_new)+ ' and '+str(end_new))


In [180]:
# print('Total number of trips in old data is ' +str(len(old_gdf_202010_to_202009['TripId'].unique())))
# print('Total number of trips in new data is ' +str(len(gdf_202010_to_202009_new['TripId'].unique())))


In [92]:
# gdf_20200930 = old_data_gdf[old_data_gdf['MST_start_date']== '2020-09-30']

In [179]:
# len(gdf_20200930['TripId'].unique())

In [94]:
trip_df = pd.read_csv('trips (1).csv', keep_default_na=True, names=header)
small_trips_df = pd.read_csv('trips.csv1.csv', keep_default_na=True, names=header)

In [178]:
print('large data total number of trips: '+ str(len(trip_df['TripId'].unique())))
print('small data total number of trips: '+ str(len(small_trips_df['TripId'].unique())))

test = len(trip_df) + 1051

large data total number of trips: 2316130
small data total number of trips: 1160372


In [ ]:
trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

large_gdf = gpd.GeoDataFrame(trip_df)
large_gdf['MST_start_datetime'] = pd.to_datetime(large_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
large_gdf['MST_end_datetime'] = pd.to_datetime(large_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

large_gdf['MST_start_date'] = large_gdf['MST_start_datetime'].apply(lambda x:x.date())
large_gdf['MST_start_time'] = large_gdf['MST_start_datetime'].apply(lambda x:x.time())

large_gdf['MST_end_date'] = large_gdf['MST_end_datetime'].apply(lambda x:x.date())
large_gdf['MST_end_time'] = large_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [117]:
small_trips_df['Startlocation'] = list(zip(trip_df.StartLocLat, small_trips_df.StartLocLon))
small_trips_df['Startlocation'] = small_trips_df['Startlocation'].apply(Point)

small_trips_df['Endlocation'] = list(zip(small_trips_df.EndLocLat, small_trips_df.EndLocLon))
small_trips_df['Endlocation'] = small_trips_df['Endlocation'].apply(Point)

# small_trips_df= small_trips_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

small_gdf = gpd.GeoDataFrame(small_trips_df)
small_gdf['MST_start_datetime'] = pd.to_datetime(small_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
small_gdf['MST_end_datetime'] = pd.to_datetime(small_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

small_gdf['MST_start_date'] = small_gdf['MST_start_datetime'].apply(lambda x:x.date())
small_gdf['MST_start_time'] = small_gdf['MST_start_datetime'].apply(lambda x:x.time())

small_gdf['MST_end_date'] = small_gdf['MST_end_datetime'].apply(lambda x:x.date())
small_gdf['MST_end_time'] = small_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [118]:
print(small_gdf['MST_start_date'].min(), small_gdf['MST_start_date'].max())
print

2020-09-30 2021-09-30


In [119]:
print(large_gdf['MST_start_date'].min(), large_gdf['MST_start_date'].max())

2020-08-31 2021-09-30


### End of latest edit